## Oyo Rooms:

In [1]:
#URL used: https://www.oyorooms.com/hotels-in-coimbatore/

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
import os
from selenium import webdriver    
path_tounzipfile='C:/Users/Srishti/Desktop/Data Science/Data Scrapping & Wrangling/geckodriver-v0.25.0-win64'
os.chdir(path_tounzipfile)

In [3]:
url_oyo='https://www.oyorooms.com/hotels-in-coimbatore/?page=4'
browser=webdriver.Firefox(path_tounzipfile)
browser.get(url_oyo)
html_oyo=browser.page_source
soup_oyo=BeautifulSoup(html_oyo,'html.parser')    

In [4]:
listings=soup_oyo.find_all('div',class_='hotelCardListing__descriptionWrapper')
len(listings)

10

In [5]:
listings[0].find_all('h3',class_='listingHotelDescription__hotelName d-textEllipsis')

[<h3 class="listingHotelDescription__hotelName d-textEllipsis" itemprop="name" title="OYO TOWNHOUSE 174 Gandhipuram">OYO TOWNHOUSE 174 Gandhipuram</h3>]

In [6]:
listings[0].find_all('span',class_='listingPrice__finalPrice')

[]

In [7]:
##Experiment with page 5:
url_oyo='https://www.oyorooms.com/hotels-in-coimbatore/?page=5'
browser=webdriver.Firefox(path_tounzipfile)
browser.get(url_oyo)
html_oyo=browser.page_source
soup_oyo=BeautifulSoup(html_oyo,'html.parser')

In [8]:
listings=soup_oyo.find_all('div',class_='hotelCardListing__descriptionWrapper')
len(listings)

0

In [29]:
listings[0].find_all('span',class_='listingPrice__finalPrice')

[]

In [10]:
#develop the full logic:
url_oyo='https://www.oyorooms.com/hotels-in-coimbatore/?page=4'
browser=webdriver.Firefox(path_tounzipfile)
browser.get(url_oyo)
html_oyo=browser.page_source
soup_oyo=BeautifulSoup(html_oyo,'html.parser')

In [11]:
listings=soup_oyo.find_all('div',class_='hotelCardListing__descriptionWrapper')
len(listings)

10

In [12]:
browser=webdriver.Firefox(path_tounzipfile)
def get_data(url):
    browser.get(url)
    html_oyo=browser.page_source
    soup_oyo=BeautifulSoup(html_oyo,'html.parser')
    listings=soup_oyo.find_all('div',class_='hotelCardListing__descriptionWrapper')
    name=[]
    address=[]
    ratings=[]
    price=[]
    
    for i in listings:

        n=i.find_all('h3',class_='listingHotelDescription__hotelName d-textEllipsis')
        if len(n)!=0:
            name.append(n[0].text)
        else:
            name.append(np.nan)

        adrs=i.find_all('span',itemprop='streetAddress')
        if len(adrs)!=0:
            address.append(adrs[0].text)
        else:
            address.append(np.nan)

        rating=i.find_all('meta',itemprop='ratingValue')
        if len(rating)!=0:
            ratings.append(rating[0]['content'])
        else:
            ratings.append(np.nan)

        pr=i.find_all('span',class_='listingPrice__finalPrice')
        if len(pr)!=0:
            price.append(pr[0].text)
        else:
            price.append(np.nan)
    data_frame=pd.DataFrame({'name':name,'address':address,'ratings':ratings,'price':price})
    return data_frame

#url_test='https://www.oyorooms.com/hotels-in-coimbatore/?page=2'
#get_data(url_test)

base_url='https://www.oyorooms.com/hotels-in-coimbatore/?page={}'
master={}
for i in range(1,6):
    base_url_list=base_url.format(i)
    master[i]=get_data(base_url_list)

In [13]:
master[1].head()

,name,address,ratings,price
0,OYO Townhouse 124 Srm Kuppakonam Pudur,"Near Tatabad, Coimbatore",4.4,₹1394
1,SilverKey Executive Stays 17281 Athithi Inn De...,"Near Aathi Maaruthi, Vetri Vinayagar Nagar, Co...",4.3,₹775
2,OYO 28079 Global Stay,"Near Singanallur Lake, Nanjappa Nagar, Coimbatore",3.9,₹1010
3,Capital O 48497 Raksha Grand,", Coimbatore",4.8,₹996
4,SilverKey Executive Stays 24155 FCI Nagar,"Near Saravanampatti Check Post, Coimbatore",4.2,₹937


In [14]:
master[5].tail()

,name,address,ratings,price


In [15]:
table=pd.concat([master[i] for i in master],axis=0)
table.tail()

,name,address,ratings,price
5,OYO 15427 Hotel Apple Grand Deluxe,"4th Extension, 100 Feet Road, Coimbatore",4,NaN
6,SPOT ON 42738 Thangam Lodge,"Near Gandhipuram Town Bu Stand, Coimbatore",4.1,NaN
7,OYO 45997 Gsn Residency Saver,"Near Skoda showroom, Coimbatore",4.3,NaN
8,OYO Flagship 35720 Sai Renu Vadavalli,"Near Arun Nagar Park, Coimbatotre",4.3,NaN
9,OYO 1531 Vels Grand Inn Hotel,"Gandhipuram, Coimbatore",4.1,NaN


In [16]:
#clean price:
table['price']=table['price'].str.replace('₹','').astype('float')
table.head()

,name,address,ratings,price
0,OYO Townhouse 124 Srm Kuppakonam Pudur,"Near Tatabad, Coimbatore",4.4,1394.0
1,SilverKey Executive Stays 17281 Athithi Inn De...,"Near Aathi Maaruthi, Vetri Vinayagar Nagar, Co...",4.3,775.0
2,OYO 28079 Global Stay,"Near Singanallur Lake, Nanjappa Nagar, Coimbatore",3.9,1010.0
3,Capital O 48497 Raksha Grand,", Coimbatore",4.8,996.0
4,SilverKey Executive Stays 24155 FCI Nagar,"Near Saravanampatti Check Post, Coimbatore",4.2,937.0


In [17]:
#Cleaning rating:
table['ratings']=table['ratings'].astype('float')
table.head()

,name,address,ratings,price
0,OYO Townhouse 124 Srm Kuppakonam Pudur,"Near Tatabad, Coimbatore",4.4,1394.0
1,SilverKey Executive Stays 17281 Athithi Inn De...,"Near Aathi Maaruthi, Vetri Vinayagar Nagar, Co...",4.3,775.0
2,OYO 28079 Global Stay,"Near Singanallur Lake, Nanjappa Nagar, Coimbatore",3.9,1010.0
3,Capital O 48497 Raksha Grand,", Coimbatore",4.8,996.0
4,SilverKey Executive Stays 24155 FCI Nagar,"Near Saravanampatti Check Post, Coimbatore",4.2,937.0
